In [ ]:
!git clone https://github.com/deepanshudashora/custom_models.git

In [ ]:
!pip3 install torch_lr_finder

In [ ]:
import config
from custom_models.YOLOv3.utils import get_loaders

import warnings
warnings.filterwarnings("ignore")

import torch 
import torch.optim as optim

import pandas as pd 
from custom_models.YOLOv3.loss import YoloLoss
loss_fn = YoloLoss()
from IPython.core.display import display
import seaborn as sn
from custom_models.YOLOv3.model import YOLOv3
from custom_models.YOLOv3.dataset import YOLODataModule
from custom_models.YOLOv3.main import YOLOTraining,check_accuracy,make_trainer
from custom_models.utils import find_max
from custom_models.YOLOv3.utils import find_max_lr, criterion
import os 
import warnings
warnings.filterwarnings("ignore")

In [ ]:
print("/kaggle/input/pascal-voc-dataset-used-in-yolov3-video/PASCAL_VOC/" + "train.csv")
train_loader, test_loader, train_eval_loader = get_loaders(
    train_csv_path="/kaggle/input/pascal-voc-dataset-used-in-yolov3-video/PASCAL_VOC/"+ "train.csv", test_csv_path="/kaggle/input/pascal-voc-dataset-used-in-yolov3-video/PASCAL_VOC/" + "test.csv"
)

In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
model = YOLOv3(num_classes=config.NUM_CLASSES).to(DEVICE)
optimizer = optim.Adam(
    model.parameters(), lr=config.LEARNING_RATE, weight_decay=config.WEIGHT_DECAY
)
scaled_anchors = (
    torch.tensor(config.ANCHORS)
    * torch.tensor(config.S).unsqueeze(1).unsqueeze(1).repeat(1, 3, 2)
).to(DEVICE)

end_lr = 2

find_max_lr(model,optimizer,criterion,train_loader,end_lr)

In [ ]:
max_epochs = 40
max_lr = 1.23E-02
learning_rate = config.LEARNING_RATE
weight_decay = config.WEIGHT_DECAY
check_val_every_n_epoch = 20
make_trainer(max_epochs,train_loader, test_loader, train_eval_loader,max_lr,
                 learning_rate,weight_decay,check_val_every_n_epoch,config,precision=16,
                 refresh_rate=10,accelerator="auto",
                 tensorboard_logs = "/kaggle/working/tf_logs/",
                 csv_logs = "/kaggle/working/csv_training_logs/"
            )